In [1]:
import tensorflow as tf
import datetime
from Models.Transformers import Transformer, ESBNTransformer, ESBNTransformerSCA
from Utils.preprocess_scan import generate_data
import tensorboard
%load_ext tensorboard
tf.config.run_functions_eagerly(False)

/Users/jonathankonig/miniforge3/envs/esbn_abstractor/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In this notebook, we compare two variants of integrating the esbn into a transformer against the standard transformer. The code for the transformer is adapted from the tensorflow blog.

In [2]:
# Generate data and get vocab sizes
train_ds, val_ds, command_vocab_len, action_vocab_len = generate_data()

2024-04-03 19:40:51.760034: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [3]:
# Define evaluation metrics
def masked_loss(label, pred):
  mask = label != 0
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
  loss = loss_object(label, pred)

  mask = tf.cast(mask, dtype=loss.dtype)
  loss *= mask

  loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
  return loss


def masked_accuracy(label, pred):
  pred = tf.argmax(pred, axis=2)
  label = tf.cast(label, pred.dtype)
  match = label == pred

  mask = label != 0

  match = match & mask

  match = tf.cast(match, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(match)/tf.reduce_sum(mask)

In [4]:
# Ordinary Transformer
num_layers = 2
d_model = 32
dff = 64
num_heads = 4
dropout_rate = 0.1

transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=command_vocab_len,
    target_vocab_size=action_vocab_len,
    dropout_rate=dropout_rate)

transformer.compile(
    loss=masked_loss,
    optimizer="Adam",
    metrics=[masked_accuracy])

log_dir = "logs/fit/scan_transformer/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
transformer.fit(train_ds,
                epochs=20,
                validation_data=val_ds,
                callbacks=[tensorboard_callback])

transformer.summary()

Epoch 1/20
458/458 [==============================] - 22s 40ms/step - loss: 0.8136 - masked_accuracy: 0.6780 - val_loss: 2.8789 - val_masked_accuracy: 0.4229
Epoch 2/20
458/458 [==============================] - 17s 36ms/step - loss: 0.5087 - masked_accuracy: 0.7769 - val_loss: 3.0768 - val_masked_accuracy: 0.4735
Epoch 3/20
458/458 [==============================] - 18s 39ms/step - loss: 0.4339 - masked_accuracy: 0.8051 - val_loss: 3.1229 - val_masked_accuracy: 0.5073
Epoch 4/20
458/458 [==============================] - 17s 37ms/step - loss: 0.3477 - masked_accuracy: 0.8513 - val_loss: 3.2294 - val_masked_accuracy: 0.5464
Epoch 5/20
458/458 [==============================] - 17s 37ms/step - loss: 0.2415 - masked_accuracy: 0.8989 - val_loss: 3.3636 - val_masked_accuracy: 0.5602
Epoch 6/20
458/458 [==============================] - 18s 40ms/step - loss: 0.1976 - masked_accuracy: 0.9181 - val_loss: 3.5235 - val_masked_accuracy: 0.5514
Epoch 7/20
458/458 [==============================] 

In [5]:
num_layers = 2
d_model = 32
dff = 64
num_heads = 4
dropout_rate = 0.1

esbn_transformer = ESBNTransformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=command_vocab_len,
    target_vocab_size=action_vocab_len,
    dropout_rate=dropout_rate)

esbn_transformer.compile(
    loss=masked_loss,
    optimizer="Adam",
    metrics=[masked_accuracy])

log_dir = "logs/fit/scan_transformer_esbn/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
esbn_transformer.fit(train_ds,
                epochs=20,
                validation_data=val_ds,
                callbacks=[tensorboard_callback])

esbn_transformer.summary()

Epoch 1/20
458/458 [==============================] - 35s 60ms/step - loss: 0.8085 - masked_accuracy: 0.6803 - val_loss: 2.7810 - val_masked_accuracy: 0.4616
Epoch 2/20
458/458 [==============================] - 28s 61ms/step - loss: 0.5051 - masked_accuracy: 0.7820 - val_loss: 2.8242 - val_masked_accuracy: 0.4950
Epoch 3/20
458/458 [==============================] - 28s 61ms/step - loss: 0.3575 - masked_accuracy: 0.8517 - val_loss: 2.7274 - val_masked_accuracy: 0.5765
Epoch 4/20
458/458 [==============================] - 28s 62ms/step - loss: 0.2430 - masked_accuracy: 0.9009 - val_loss: 2.7162 - val_masked_accuracy: 0.5822
Epoch 5/20
458/458 [==============================] - 29s 63ms/step - loss: 0.1899 - masked_accuracy: 0.9233 - val_loss: 2.5777 - val_masked_accuracy: 0.6329
Epoch 6/20
458/458 [==============================] - 29s 63ms/step - loss: 0.1551 - masked_accuracy: 0.9376 - val_loss: 2.8128 - val_masked_accuracy: 0.6479
Epoch 7/20
458/458 [==============================] 

In [6]:
num_layers = 2
d_model = 32
dff = 64
num_heads = 4
dropout_rate = 0.1

esbn_transformer_sca = ESBNTransformerSCA(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=command_vocab_len,
    target_vocab_size=action_vocab_len,
    dropout_rate=dropout_rate)

esbn_transformer_sca.compile(
    loss=masked_loss,
    optimizer="Adam",
    metrics=[masked_accuracy])

log_dir = "logs/fit/scan_transformer_esbn_sca/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
esbn_transformer_sca.fit(train_ds,
                epochs=20,
                validation_data=val_ds,
                callbacks=[tensorboard_callback])

esbn_transformer_sca.summary()

Epoch 1/20
458/458 [==============================] - 34s 57ms/step - loss: 0.7831 - masked_accuracy: 0.6858 - val_loss: 2.7328 - val_masked_accuracy: 0.4926
Epoch 2/20
458/458 [==============================] - 27s 58ms/step - loss: 0.4942 - masked_accuracy: 0.7856 - val_loss: 2.9598 - val_masked_accuracy: 0.5116
Epoch 3/20
458/458 [==============================] - 26s 57ms/step - loss: 0.3583 - masked_accuracy: 0.8456 - val_loss: 2.8594 - val_masked_accuracy: 0.5700
Epoch 4/20
458/458 [==============================] - 27s 59ms/step - loss: 0.2245 - masked_accuracy: 0.9074 - val_loss: 2.8846 - val_masked_accuracy: 0.5834
Epoch 5/20
458/458 [==============================] - 26s 58ms/step - loss: 0.1597 - masked_accuracy: 0.9354 - val_loss: 2.9784 - val_masked_accuracy: 0.5838
Epoch 6/20
458/458 [==============================] - 27s 58ms/step - loss: 0.1244 - masked_accuracy: 0.9509 - val_loss: 2.9334 - val_masked_accuracy: 0.6213
Epoch 7/20
458/458 [==============================] 

In [7]:
esbn_transformer_sca.summary()

Model: "esbn_transformer_sca"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_2 (Encoder)         multiple                  42784     
                                                                 
 decoder_3 (Decoder)         multiple                  76288     
                                                                 
 decoder_4 (Decoder)         multiple                  76288     
                                                                 
 esbn_encoder_cross_attentio  multiple                 52323     
 n (ESBNEncoderCrossAttentio                                     
 n)                                                              
                                                                 
 dense_46 (Dense)            multiple                  650       
                                                                 
Total params: 248,333
Trainable params: 248,33

In [9]:
%tensorboard --logdir logs/fit

Reusing TensorBoard on port 6006 (pid 58338), started 0:04:30 ago. (Use '!kill 58338' to kill it.)